In [2]:
from ia_model.data_preparation import load_data, prepare_data, split_data
# from ia_model.model import  train_model, evaluate_model, save_model, build_dynamic_model
from ia_model.utils import ia_plot_history
from ia_model.model_final import evaluate_combinations, build_best_model
import joblib


In [11]:
# Configuration parameters for the model and data preparation
FILE_PATH = 'aggregated_data4.csv' #This data can be download from https://rcavieses.pythonanywhere.com/
FEATURE_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA','SST','PESO DESEMBARCADO_KILOGRAMOS']#Change this if you want to add input variables
TARGET_COL = 'PESO DESEMBARCADO_KILOGRAMOS'
CATEGORICAL_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA']
NUMERIC_COL = ['PESO DESEMBARCADO_KILOGRAMOS','SST']
steps_forecast = 12
time_steps= 12 # number of steps that lstm forecast, change this will be errors on scoore metrics ;
# Example 
neurons_list = [20,30,40]
layers_list = [2,3,4]
activations_list = ['relu']  # List of activation functions to test, possible choice ['relu', 'sigmoid','tanh'] 
EPOCHS = 50
BATCH_SIZE = 30
MODEL_PATH = 'mi_modelo.h5'
SCALER_X_PATH = 'scaler_X.pkl'
SCALER_Y_PATH = 'scaler_Y.pkl'




In [7]:
# Load and prepare data
df = load_data(str(FILE_PATH))
X, y, scaler_x,scaler_y, encoder , n_features = prepare_data(df, TARGET_COL, time_steps,CATEGORICAL_COL,NUMERIC_COL)
X_train, X_test, y_train, y_test = split_data(X, y)

In [12]:
# Build model and test different combination to find the best combination
best_configuration, best_mse, best_r2 = evaluate_combinations(X_train, y_train, X_test, y_test, time_steps, n_features, neurons_list, layers_list, activations_list, epochs=50, batch_size=32)
print(f"Best configuration: {best_configuration}\nMSE: {best_mse}, R^2: {best_r2}")

1668/1668 [==============================] - 6s 3ms/step
Best configuration: {'layers': 4, 'neurons': 40, 'activation': 'relu', 'MSE': 3.217697505078438e-05, 'R2': 0.06952714420618267}
MSE: 3.217697505078438e-05, R^2: 0.06952714420618267


In [4]:
# Build, Train, Evaluate the model using best parameter
best_configuration = {'layers': 3, 'neurons': 32, 'activation': 'relu'}  # gives r2 of 0.32
model, mse, r2 = build_best_model(time_steps, n_features, best_configuration, epochs=50, batch_size=32)


In [ ]:
ia_plot_history(history)

In [ ]:
save_model(model,'mi_modelo.h5')
joblib.dump(scaler_x, 'scaler_X.pkl')  # Saves the scaler
joblib.dump(scaler_y, 'scaler_Y.pkl')  # Saves the scaler
joblib.dump(encoder, 'encoder_filename.pkl')  # Saves the encoder